In [2]:
#Imports:
from ncps.wirings import AutoNCP
from ncps.torch import CfC
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from numpy import genfromtxt
import numpy as np
import torch
import torch.utils.data as data
import matplotlib as plt
import torch.nn as nn

torch.set_float32_matmul_precision("high")

In [3]:
#PROPER FORMATTING
#Time sequences are 10 timepoints (Messages) with 7 features per message.
#Organized by car.

#Current Simulation File
dataFile = 'Data/CfCMultiExtension/DoS_0709.csv'

dataSet = genfromtxt(dataFile, delimiter=',')
batchSize = 64
# Ceate dataloader and fill with (BSM, attk#). Expanding to add 0th dimension for batches.
# Batch size should be 64 for the low density simulations and 128 for high density simulations.
# No shuffle to keep batches on same vehicle.
# Num_workers is set to = num CPU cores
dataSet[0:-1,:] = dataSet[1:,:] # Get rid of the first null value of the dataset
print(dataSet.shape)
# count subsets per vehicle
unq, counts = np.unique(dataSet[:, 2], return_counts = True)
sender = 0
lastSenderCount = 0
newData = []
# Organize dataset into sets of 10 messages by sender
while sender < counts.shape[0]:
    # Loop through sender
    index = 0
    while index < counts[sender] - 10:
        # Loop through messages from sender
        newData.append(dataSet[lastSenderCount+index:lastSenderCount +index+10])
        index += 5
    sender += 1
    lastSenderCount = counts[sender-1]
dataSet = torch.tensor(newData)
len = dataSet.shape[0]
trainPerc = 80
# Create new arrays per vehicle for federated learning
splits = np.split(dataSet, np.cumsum(counts)[:-1])
# Create seperate datasets for testing and training, using Train Percentage as metric for split
trainDataIn = torch.Tensor(dataSet[:int(len*(trainPerc/100)),:,1:10]).float()
trainDataOut = torch.Tensor(np.int_(dataSet[:int(len*(trainPerc/100)),:,11])).long()
testDataIn = torch.Tensor(dataSet[int(len*(trainPerc/100)):,:,1:10]).float()
testDataOut = torch.Tensor(np.int_(dataSet[int(len*(trainPerc/100)):,:,11])).long()
newsetIn = []
newsetOut = []
# Create dataset of 1/100th of the entries for quicker testing during development
for index in range(0,int(len * (trainPerc/100))):
    if not (int(index/10) % 100):
        newsetIn.append(dataSet[index,:,1:10])
        newsetOut.append((dataSet[index,:,11]))
testingIn = torch.Tensor(np.array(newsetIn)).float()
testingOut = torch.Tensor(np.array(newsetOut)).long()
# Create Dataloaders for all the datasets
dataLoaderTrain = data.DataLoader(data.TensorDataset(trainDataIn, trainDataOut), batch_size=batchSize, shuffle=False, num_workers=16)
dataLoaderTest = data.DataLoader(data.TensorDataset(testDataIn, testDataOut), batch_size=batchSize, shuffle=False, num_workers=16)
testingDataLoader = data.DataLoader(data.TensorDataset(testingIn, testingOut), batch_size=batchSize, shuffle = False, num_workers=16)

(4957201, 12)


C:\Users\will\AppData\Local\Temp\ipykernel_23632\2229332239.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  dataSet = torch.tensor(newData)


In [4]:
print(trainDataIn.shape)
print(trainDataOut.shape)
print(testingOut.shape)
print(testingIn.shape)
print(trainDataIn[-1])
print(trainDataOut[1])
print(testDataIn[15])
print(testingOut[0])
print(dataSet[0:2])
print(dataSet[90].shape)


torch.Size([786645, 10, 9])
torch.Size([786645, 10])
torch.Size([7870, 10])
torch.Size([7870, 10, 9])
tensor([[9.3000e+01, 3.3000e+01, 2.5286e+04, 1.3507e+01, 1.0802e+01, 3.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.3000e+01, 3.3000e+01, 2.5286e+04, 1.3507e+01, 1.0802e+01, 4.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.3000e+01, 3.3000e+01, 2.5286e+04, 1.3327e+01, 1.0667e+01, 1.0000e+00,
         8.6027e+00, 6.3077e+00, 4.3233e-01],
        [9.3000e+01, 3.3000e+01, 2.5286e+04, 1.3327e+01, 1.0667e+01, 2.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.3000e+01, 3.3000e+01, 2.5287e+04, 1.3327e+01, 1.0667e+01, 3.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.3000e+01, 3.3000e+01, 2.5287e+04, 1.3327e+01, 1.0667e+01, 4.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.3000e+01, 3.3000e+01, 2.5287e+04, 1.3129e+01, 1.0573e+01, 1.0000e+00,
         8.3659e+00, 6.2256e+00, 4.4146e-01],
        [

In [5]:
# Creating Learner
class CfCLearner(pl.LightningModule):
    def __init__(self, model, lr):
        super().__init__()
        self.model = model
        self.lr = lr
        self.lossFunc = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("trainLoss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        print(f"output: {output.shape}")
        print(f"target: {target.shape}")
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("valLoss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        # Using AdamW optomizer based on info from paper
        return torch.optim.AdamW(self.model.parameters(), lr = self.lr)

In [6]:
class Modena(nn.Module): 
    # CfC with feed-forward layer to classify at end.
    def __init__(self, inputSize, unitNum, motorNum, outputDim, batchFirst = True):
        super().__init__()
        # Create NCP wiring for CfC
        wiring = AutoNCP(unitNum, motorNum)
        # Create CfC model with inputs and wiring
        self.cfc = CfC(inputSize, wiring, batch_first=batchFirst)
        # Create feed-forward layer
        self.fF = nn.Linear(motorNum, outputDim)
    
    def forward(self, batch, hidden = None):
        batch, hidden = self.cfc(batch, hidden) # Pass inputs through CfC
        out = nn.functional.relu(self.fF(batch)) # pass through FeedForward Layer, then make 0 minimum
        return out, hidden # Return the guess and the hidden state

In [7]:
# OBU module class to organize
class OBU():
    def __init__(self, inputSize = 9, units = 20, motors = 8, outputs = 20, epochs = 10, lr = 0.001, gpu = False):
        self.lr = lr
        self.epochs = epochs
        self.gpu = gpu
        self.model = Modena(inputSize, units, motors, outputs)
        self.learner = CfCLearner(self.model, lr) # tune units, lr
        self.trainer = pl.Trainer(
            logger = CSVLogger('log'), # Set ouput destination of logs, logging accuracy every 50 steps
            max_epochs = epochs, # Number of epochs to train for
            gradient_clip_val = 1, # This is said to stabilize training, but we should test if that is true
            accelerator = "gpu" if gpu else "cpu" # Using the GPU to run training or not
            )
    
    def fit(self, dataLoader):
        # calling built in fit function
        return self.trainer.fit(self.learner, dataLoader)
    
    # Function to run model through a testing dataset and calculate accuracy. Can be expanded to give more metrics and more useful metrics.
    def test(self, dataIn, dataOut):
        # Put input data through model and determine classification
        with torch.no_grad():
            outs = np.asarray(self.model(dataIn)[0])
        outs = torch.from_numpy(outs)
        # Get the label with the maximum confidence for determining classification
        print(outs.shape)
        _, res = torch.max(outs, 2)
        countR = 0
        numZero = 0
        tot = outs.shape[0]
        total = 0
        for i in range(0, tot):
            # Loop through sequences of 10 each
            for t in range(0, res[i].shape[0]):
                # Loop through the sub-sequences
                if res[i,t] == dataOut[i,t]:
                    # Check if label is correct, and add to count right accordingly
                    countR += 1
                if dataOut[i,t] == 0:
                    # If the label is zero, increment the count of zeroes to determine if model is just outputting zeroes
                    numZero += 1
                total += 1
        # Calculate percent correct and percent zero
        perc = (countR/total) * 100
        percZero = (numZero/total) * 100
        print("Model got " + str(countR) + "/" + str(total) + " right. Accuracy of " + str(perc) + "%")
        print(str(percZero) + "% Zeroes.")
        return countR, total, perc, percZero
    
    def testStep(self, dataLoader):
        self.learner.validation_step(next(iter(dataLoader)), 0)


In [15]:
testOBU = OBU(
    inputSize = 9,  # Number of features per BSM
    units = 20, # Number of hidden cells
    motors = 8, # Number of motor neurons
    outputs = 20, # Number of possible labels
    epochs = 100,
    lr = 0.001,
    gpu = True
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
testOBU.testStep(dataLoaderTest)

output: torch.Size([64, 20, 10])
target: torch.Size([64, 10])


c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\core\module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


In [12]:
print("Before Training:")
testOBU.test(testDataIn, testDataOut)

Before Training:
torch.Size([196662, 10, 20])
Model got 0/1966620 right. Accuracy of 0.0%
39.65916140382993% Zeroes.


(0, 1966620, 0.0, 39.65916140382993)

In [13]:
testOBU.fit(testingDataLoader)

c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.7 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.4 K     Trainable params
280       Non-trainable params
1.7 K     Total params
0.007     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode
c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 19: 100%|██████████| 123/123 [00:22<00:00,  5.40it/s, v_num=231, trainLoss=0.675]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 123/123 [00:22<00:00,  5.39it/s, v_num=231, trainLoss=0.675]


In [14]:
print("After Training:")
countR, tot, perc, percZero = testOBU.test(testDataIn, testDataOut)

After Training:
torch.Size([196662, 10, 20])
Model got 1186675/1966620 right. Accuracy of 60.34083859617007%
39.65916140382993% Zeroes.
